In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format

In [29]:
df = pd.read_csv('../../data/processed/df_prep.csv')

In [17]:
df

,totalTravelDistance,totalFare,travelDurationHours,numSegments,daysUntilDeparture,startingAirport_encoded,destinationAirport_encoded,isNonStop_encoded,isBasicEconomy_encoded,isRefundable_encoded,cabinType_encoded,timeBlock_encoded
0,2281.0,317.60,6.633333,2,2,12,0,0,0,0,1,0
1,2175.0,487.60,9.050000,2,2,12,0,0,0,0,1,2
2,2393.0,526.11,12.483333,3,2,12,0,0,0,0,1,0
3,2393.0,531.71,12.483333,3,2,12,0,0,0,0,1,0
4,2150.0,534.01,7.866667,2,2,12,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
12560375,3039.0,770.60,9.966667,2,15,11,15,0,0,0,1,1
12560376,3670.0,801.60,11.666667,2,15,11,15,0,0,0,1,0
12560377,3670.0,813.60,11.233333,2,15,11,15,0,0,0,1,1
12560378,3670.0,813.60,11.400000,2,15,11,15,0,0,0,1,0


In [26]:
df.describe()

,totalTravelDistance,totalFare,travelDurationHours,numSegments,daysUntilDeparture,startingAirport_encoded,destinationAirport_encoded,isNonStop_encoded,isBasicEconomy_encoded,isRefundable_encoded,cabinType_encoded,timeBlock_encoded
count,12560308.00,12560308.00,12560308.00,12560308.00,12560308.00,12560308.00,12560308.00,12560308.00,12560308.00,12560308.00,12560308.00,12560308.00
mean,1569.61,382.06,6.80,1.78,27.56,7.53,7.51,0.30,0.06,0.00,1.01,0.83
std,841.49,209.72,3.62,0.58,17.36,4.67,4.66,0.46,0.24,0.00,0.15,0.98
min,97.00,23.97,0.02,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,862.00,228.60,3.97,1.00,12.00,3.00,3.00,0.00,0.00,0.00,1.00,0.00
50%,1392.00,353.60,6.47,2.00,26.00,8.00,8.00,0.00,0.00,0.00,1.00,1.00
75%,2376.00,498.19,9.08,2.00,42.00,11.00,11.00,1.00,0.00,0.00,1.00,1.00
max,4430.00,4017.60,23.95,4.00,60.00,15.00,15.00,1.00,1.00,1.00,4.00,3.00


In [25]:
df = df[df['totalFare'] <= 4500]

# Model 

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [30]:
X = df.drop(['totalFare'], axis=1)
y = df[['totalFare']]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

## Xgboost

In [31]:
from xgboost import XGBRegressor

# Initialise the XGBoost regressor with chosen hyperparameters
xgb_regressor = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=-1)

# Train the model
xgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")


Mean Squared Error: 16781.263744673426
Mean Absolute Error: 89.60630029751381
R^2 Score: 0.6210252046585083


In [32]:
# Initialise the XGBoost regressor with best hyperparameters
xgb_regressor = XGBRegressor(
    n_estimators=300, 
    learning_rate=0.1, 
    max_depth=6, 
    min_child_weight=5,
    subsample=0.8, 
    colsample_bytree=1.0, 
    gamma=0,
    random_state=42, 
    n_jobs=-1
)

# Train the model
xgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")


Mean Squared Error: 14685.774580131236
Mean Absolute Error: 82.72984209374475
R^2 Score: 0.6683480739593506


In [4]:
df_filtered = df[df['totalFare'] <= 4500]

In [5]:
df_filtered

,totalFare,totalTravelDistance,dep_day,dep_month,dep_hour,dep_minute,ar_day,ar_month,ar_hour,ar_minute,...,dest__LGA,dest__MIA,dest__OAK,dest__ORD,dest__PHL,dest__SFO,coach,business,first,premium coach
0,317.60,2281.0,13,5,18,42,14,5,1,20,...,0,0,0,0,0,0,2,0,0,0
1,487.60,2175.0,14,5,0,18,14,5,9,21,...,0,0,0,0,0,0,2,0,0,0
2,526.11,2393.0,13,5,14,16,14,5,2,45,...,0,0,0,0,0,0,3,0,0,0
3,531.71,2393.0,13,5,14,16,14,5,2,45,...,0,0,0,0,0,0,3,0,0,0
4,534.01,2150.0,14,5,1,58,14,5,9,50,...,0,0,0,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4784420,480.20,3670.0,21,5,0,48,21,5,16,39,...,0,0,0,0,0,1,2,0,0,0
4784421,492.20,2685.0,20,5,10,45,20,5,23,18,...,0,0,0,0,0,1,2,0,0,0
4784422,507.10,2623.0,20,5,19,25,21,5,5,26,...,0,0,0,0,0,1,3,0,0,0
4784423,557.20,2731.0,20,5,12,0,21,5,1,54,...,0,0,0,0,0,1,2,0,0,0


In [7]:
X = df_filtered.drop(['totalFare'], axis=1)
y = df_filtered[['totalFare']]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [8]:
from xgboost import XGBRegressor

# Initialise the XGBoost regressor with chosen hyperparameters
xgb_regressor = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=-1)

# Train the model
xgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 17369.952421179656
Mean Absolute Error: 91.56477153207611
R^2 Score: 0.6254907846450806


In [12]:
df_drop_lay = df.drop(['layover_h', 'layover_m'], axis=1)

In [13]:
X = df_drop_lay.drop(['totalFare'], axis=1)
y = df_drop_lay[['totalFare']]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [14]:
from xgboost import XGBRegressor

# Initialise the XGBoost regressor with chosen hyperparameters
xgb_regressor = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=-1)

# Train the model
xgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 17808.18875462251
Mean Absolute Error: 92.26080788208837
R^2 Score: 0.6205587387084961


In [15]:
df_drop_dur = df.drop(['duration_days', 'duration_hours', 'duration_minutes'], axis=1)

In [16]:
X = df_drop_dur.drop(['totalFare'], axis=1)
y = df_drop_dur[['totalFare']]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [17]:
from xgboost import XGBRegressor

# Initialise the XGBoost regressor with chosen hyperparameters
xgb_regressor = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=-1)

# Train the model
xgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 17670.462834111935
Mean Absolute Error: 91.5893745242749
R^2 Score: 0.6234933137893677


In [18]:
df_both = df.drop(['duration_days', 'duration_hours', 'duration_minutes', 'layover_h', 'layover_m'], axis=1)

In [19]:
X = df_drop_dur.drop(['totalFare'], axis=1)
y = df_drop_dur[['totalFare']]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [20]:
from xgboost import XGBRegressor

# Initialise the XGBoost regressor with chosen hyperparameters
xgb_regressor = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, n_jobs=-1)

# Train the model
xgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 17670.462834111935
Mean Absolute Error: 91.5893745242749
R^2 Score: 0.6234933137893677


# Hyperprarmeter turning

In [34]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 300],                
    'learning_rate': [0.05, 0.1],              
    'max_depth': [3, 6],                       
    'subsample': [0.8, 1.0],                   
    'colsample_bytree': [0.8, 1.0],            
    'gamma': [0, 0.1],                         
    'min_child_weight': [1, 5]                
}


# Initialize the XGBRegressor
xgb_regressor = XGBRegressor(random_state=42)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_regressor,
    param_distributions=param_grid,
    n_iter=50,  # Number of parameter settings sampled
    scoring='neg_mean_squared_error',  # Optimization metric
    cv=3,  # 5-fold cross-validation
    verbose=1,
    n_jobs=-1,  # Use all available cores
    random_state=42
)

# Fit RandomizedSearchCV to the data
random_search.fit(X_train, y_train)

# Get the best model and parameters
best_xgb = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# Make predictions on the test set using the best model
y_pred = best_xgb.predict(X_test)

# Evaluate the tuned model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

Fitting 3 folds for each of 50 candidates, totalling 150 fits


KeyboardInterrupt: 